# Imports

In [ ]:
from scipy.io import loadmat
import numpy as np

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import BackendSampler
from qiskit.algorithms.state_fidelities import ComputeUncompute  # type: ignore
from qiskit_ionq import IonQProvider

from feature_map import PetersFeatureMap

# Initialization & Data Selection

In [ ]:
data = loadmat("FeGaPd_full_data_220104a.mat")

# x ray diffraction data
xrd = data["X"][:, 631:1181]

train_indices = np.array(
    [
        25,
        68,
        112,
        161,
        37,
        91,
        136,
        184,
        77,
        85,
        128,
        224,
        168,
        229,
        220,
        233,
        195,
        210,
        256,
        264,
    ]
)

# Pre-process the XRD data

In [ ]:
# kernel hyperparameters
num_qubits = 25
normalize_scale = 0.001

num_features = 6 * num_qubits

theta_indices = np.linspace(
    start=0, stop=xrd.shape[1] - 1, num=num_features, dtype=np.int16
)

# subsample the x-ray diffraction spectra
xrd_sample = xrd[np.ix_(train_indices, theta_indices)]

# logistically normalize the data
xrd_sample = 1 / (1 + np.exp(-normalize_scale * xrd_sample))
xrd_sample /= np.sum(xrd_sample)

# normalize the XRD samples to within [0, pi]
min = np.min(xrd_sample)
max = np.max(xrd_sample)

normalized_xrd_sample = np.pi * (xrd_sample - min) / (max - min)

# Instantiate Kernel

In [ ]:
feature_map = PetersFeatureMap(feature_dimension=num_features, reps=1)

with open("api_key.txt") as f:
    api_key = f.read()

provider = IonQProvider(api_key)

# ionq_backend = provider.get_backend('ionq_qpu.aria')
ionq_backend = provider.get_backend("ionq_simulator")

sampler = BackendSampler(backend=ionq_backend, options={"noise_model": "aria-1"})

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

# kernel = FidelityQuantumKernel(feature_map=feature_map)

# Evaluate Data

In [ ]:
# NOTE: will hang until IonQ job finishes; okay to interrupt once jobs are submitted
kernel_matrix = kernel.evaluate(normalized_xrd_sample)